## LLM Experimentation with DB + LangChain
This notebook uses only Redshift DB. And also removes the usage of `create_sql_query_chain` from LangChain for more flexibility. The LLM also outputs the summary of the input query in addition to the actual SQL statement.

### TODO:
- No (implicit) database execution
- No need to use database other than the schema (Remove SQLDatabase from langchain)
- Add a paraphrasing the question as an output

In [ ]:
# !pip install jupyter-black
# !pip install langchain -q
# !pip install langchain-experimental -q
# !pip install boto3 --upgrade -q
# !pip install s3fs --upgrade -q

# !pip install sqlalchemy-redshift psycopg2-binary -q --upgrade
# !pip install sqlalchemy --upgrade -q

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import jupyter_black

jupyter_black.load()

In [ ]:
import pandas as pd
import os

from tqdm import tqdm

tqdm.pandas()

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate


from langchain.schema import StrOutputParser
from langchain.schema.runnable import Runnable, RunnableParallel

# from langchain.chains import create_sql_query_chain

from config import *
from utils import *

In [ ]:
# Load input data
df_qa = pd.read_csv(QA_DATA_PATH)

# Remove all gpt4 columns
gpt4_cols = [col for col in df_qa.columns if "gpt4" in col]
df_qa.drop(columns=gpt4_cols, inplace=True)
print(df_qa.shape)
df_qa.head()

In [ ]:
# Count number of executable SQLs
df_qa.query_corrected_with_list_executed.value_counts()

In [ ]:
# Get tables info with sample rows
tables_info = get_db_tables_info()

print(tables_info[:200])

In [ ]:
# Get column descriptions in xml format
col_descriptions = get_column_descriptions()
print(col_descriptions[:200])

In [ ]:
import boto3

bedrock = boto3.client("bedrock")
# bedrock.list_foundation_models(byProvider="anthropic")

In [ ]:
# Create LLM
llm = get_bedrock_llm(model_id=MODEL_ID, verbose=True, model_config=MODEL_CONFIG)

# Create Prompt from template
prompt = PromptTemplate.from_template(PROMPT)

# Create DB chain
db_chain = LLMChain(llm=llm, prompt=prompt, verbose=False)

In [ ]:
prompt

In [ ]:
# Let's try with the first query
# question = "select the inpatients who receive hospitalization because of diabetes of code list X"  # df_qa.iloc[0]["question_with_list"]
# question = "select the outpatients who are diagnosed with disease in code list X and are sent to hospital thereafter"
question = "select the outpatients who are diagnosed with diabetes of code list X"
inputs = {
    "question": question,
    "db_schema_name": DB_SCHEMA_NAME,
    "table_info": tables_info,
    "columns_description": col_descriptions,
}

response = db_chain.run(inputs)
print(response)

In [ ]:
sql = extract_xml(tags=["sql"], text=response)
summary = extract_xml(tags=["summary"], text=response)

In [ ]:
print(sql)

In [ ]:
print(summary)

### Run for all examples

In [ ]:
inputs = {
    "question": "<<question here>>",
    "db_schema_name": DB_SCHEMA_NAME,
    "table_info": tables_info,
    "columns_description": col_descriptions,
}
df_qa = df_qa.progress_apply(get_sql, args=(db_chain, inputs), axis=1)

In [ ]:
df_qa.head()

In [ ]:
# Display GT and predicted SQL
_ = df_qa.apply(
    print_gt_and_pred_sql, args=("query_corrected_with_list", "sql_predicted"), axis=1
)

In [ ]:
## Save Outputs
df_qa.to_csv(OUTPUT_PATH, index=False)